<a href="https://colab.research.google.com/github/spindouken/atlas-machine_learning/blob/main/reinforcement_learning%20/temporal_difference/Temporal_Difference.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Temporal Difference

## 0. Monte Carlo
<br>
Write the function def monte_carlo(env, V, policy, episodes=5000, max_steps=100, alpha=0.1, gamma=0.99):<br> that performs the Monte Carlo algorithm:<br>

env is the openAI environment instance<br>
V is a numpy.ndarray of shape (s,) containing the value estimate<br>
policy is a function that takes in a state and returns the next action to take<br>
episodes is the total number of episodes to train over<br>
max_steps is the maximum number of steps per episode<br>
alpha is the learning rate<br>
Returns: V, the updated value estimate<br>
```
$ ./0-main.py
[[ 0.81    0.9     0.4783  0.4305  0.3874  0.4305  0.6561  0.9   ]
 [ 0.9     0.729   0.5905  0.4783  0.5905  0.2824  0.2824  0.3874]
 [ 1.      0.5314  0.729  -1.      1.      0.3874  0.2824  0.4305]
 [ 1.      0.5905  0.81    0.9     1.     -1.      0.3874  0.6561]
 [ 1.      0.6561  0.81   -1.      1.      1.      0.729   0.5314]
 [ 1.     -1.     -1.      1.      1.      1.     -1.      0.9   ]
 [ 1.     -1.      1.      1.     -1.      1.     -1.      1.    ]
 [ 1.      1.      1.     -1.      1.      1.      1.      1.    ]]
$
```
Repo:

GitHub repository: atlas-machine_learning<br>
Directory: reinforcement_learning/temporal_difference<br>
File: 0-monte_carlo.py<br>



0-monte_carlo.py

In [ ]:
import numpy as np


def generate_episode(env, policy, max_steps):
    """
    generate an episode by following the given policy

    params:
    env: the OpenAI Gym environment instance
    policy: a function that takes in a state and returns the next action
    max_steps: the maximum number of steps in the episode

    returns a list of tuples, each containing: (state, reward)
    """
    episode_data = []
    state = env.reset()  # start a new episode and get the initial state

    for step in range(max_steps):
        action = policy(state)  # determine the action based on the current state and policy
        next_state, reward, terminate, _ = env.step(action)  # apply the action to the environment to get the next state and reward
        episode_data.append((state, reward))  # store the state and reward in the episode data

        if terminate or step > max_steps:
            break  # end the episode if the environment says it's done

        state = next_state  # move to the next state

    return episode_data

def monte_carlo(env, V, policy, episodes=5000, max_steps=100, alpha=0.1, gamma=0.99):
    """
    performs the Monte Carlo algorithm

    params:
    env: the OpenAI Gym environment instance
    V: numpy array of shape (s,) containing the value estimate
    policy: a function that takes in a state and returns the next action
    episodes: total number of episodes to train over
    max_steps: maximum number of steps per episode
    alpha: the learning rate
    gamma: the discount rate

    returns V, the updated value estimate
    """
    for episode in range(episodes):
        episode_data = generate_episode(env, policy, max_steps)  # generate an episode data using the policy
        episode_data = np.array(episode_data, dtype=int)  # convert episode data to numpy array for efficient processing

        G = 0
        # calculate return for each state in the episode, starting from the end
        for s, r in episode_data[::-1]:
            G = (gamma * G) + r  # update return with discount factor
            # first visit monte carlo (update value only for first visit of the state in the episode)
            if s not in episode_data[:episode, 0]:
                V[s] += alpha * (G - V[s])  # update value estimate

    return V.round(2)  # return the updated value estimates, rounded for readability


if __name__ == "__main__":
    import gym
    import numpy as np
    np.random.seed(0)

    env = gym.make('FrozenLake8x8-v1')
    LEFT, DOWN, RIGHT, UP = 0, 1, 2, 3

    def policy(s):
        p = np.random.uniform()
        if p > 0.5:
            if s % 8 != 7 and env.desc[s // 8, s % 8 + 1] != b'H':
                return RIGHT
            elif s // 8 != 7 and env.desc[s // 8 + 1, s % 8] != b'H':
                return DOWN
            elif s // 8 != 0 and env.desc[s // 8 - 1, s % 8] != b'H':
                return UP
            else:
                return LEFT
        else:
            if s // 8 != 7 and env.desc[s // 8 + 1, s % 8] != b'H':
                return DOWN
            elif s % 8 != 7 and env.desc[s // 8, s % 8 + 1] != b'H':
                return RIGHT
            elif s % 8 != 0 and env.desc[s // 8, s % 8 - 1] != b'H':
                return LEFT
            else:
                return UP


    V = np.where(env.desc == b'H', -1, 1).reshape(64).astype('float64')
    np.set_printoptions(precision=2)
    env.seed(0)
    print(monte_carlo(env, V, policy).reshape((8, 8)))

[[ 0.9   0.73  0.66  0.73  0.9   0.9   0.59  0.53]
 [ 0.59  0.66  0.73  0.81  0.66  0.39  0.48  0.39]
 [ 0.66  0.25  0.35 -1.    1.    0.48  0.43  0.43]
 [ 0.9   0.43  0.28  0.59  0.9  -1.    0.48  0.48]
 [ 1.    0.73  0.59 -1.    1.    1.    0.73  0.73]
 [ 1.   -1.   -1.    1.    1.    1.   -1.    0.9 ]
 [ 1.   -1.    1.    1.   -1.    1.   -1.    1.  ]
 [ 1.    1.    1.   -1.    1.    1.    1.    1.  ]]


## 1. TD(λ)
<br>
Write the function def td_lambtha(env, V, policy, lambtha, episodes=5000, max_steps=100, alpha=0.1, gamma=0.99): that performs the TD(λ) algorithm:

env is the openAI environment instance<br>
V is a numpy.ndarray of shape (s,) containing the value estimate<br>
policy is a function that takes in a state and returns the next action to take<br>
lambtha is the eligibility trace factor<br>
episodes is the total number of episodes to train over<br>
max_steps is the maximum number of steps per episode<br>
alpha is the learning rate<br>
gamma is the discount rate<br>
Returns: V, the updated value estimate<br>

$ ./1-main.py
<br>
```
[[ 0.5314  0.5905  0.3138  0.3138  0.6561  0.9     0.81    0.9   ]
 [ 0.5314  0.5905  0.4783  0.6561  0.5905  0.6561  0.6561  0.5314]
 [ 0.6561  0.729   0.5905 -1.      0.9     0.9     0.5905  0.3874]
 [ 0.729   0.81    0.81    0.9     1.     -1.      0.5314  0.4305]
 [ 0.5905  0.6561  0.81   -1.      1.      1.      0.729   0.4783]
 [ 0.9    -1.     -1.      1.      1.      1.     -1.      0.81  ]
 [ 1.     -1.      1.      1.     -1.      1.     -1.      1.    ]
 [ 0.9     0.81    1.     -1.      1.      1.      1.      1.    ]]
 ```

In [ ]:
import numpy as np

def td_lambtha(env, V, policy, lambtha, episodes=5000, max_steps=100, alpha=0.1, gamma=0.99):
    """
    env is the openAI environment instance

    params:
    V: numpy.ndarray of shape (s,) containing the value estimate
    polic: function that takes in a state and returns the next action to take
    lambtha: eligibility trace factor
    episodes: total number of episodes to train over
    max_steps: maximum number of steps per episode
    alpha: learning rate
    gamma: discount rate

    Returns: V, the updated value estimate
    """
    for episode in range(episodes):
        state = env.reset()  # reset environment to start a new episode
        eligibilityTraces = np.zeros_like(V)

        for step in range(max_steps):
            action = policy(state)  # determine action based on policy
            next_state, reward, done, info = env.step(action)  # take action

            # calculate TD Error
            TD_error = reward + gamma * V[next_state] - V[state]
            # print(f"Episode {episode+1}, Step {step+1}, State: {state}, Action: {action}, Reward: {reward}, New State: {next_state}, TD Error: {TD_error}")

            # upd elegibility trace
            eligibilityTraces[state] += 1

            # update value estimate
            V += alpha * TD_error * eligibilityTraces
            # print(f"Updated Value Function: {V}")

            # eligibility trace decay
            eligibilityTraces *= gamma * lambtha
            # print(f"Updated Eligibility Trace: {eligibilityTraces}")

            if done:
                # print(f"Episode {episode+1} finished after {step+1} steps.")
                break
            state = next_state

    return V

if __name__ == "__main__":
    import gym
    import numpy as np

    np.random.seed(0)

    env = gym.make('FrozenLake8x8-v1')
    LEFT, DOWN, RIGHT, UP = 0, 1, 2, 3

    def policy(s):
        p = np.random.uniform()
        if p > 0.5:
            if s % 8 != 7 and env.desc[s // 8, s % 8 + 1] != b'H':
                return RIGHT
            elif s // 8 != 7 and env.desc[s // 8 + 1, s % 8] != b'H':
                return DOWN
            elif s // 8 != 0 and env.desc[s // 8 - 1, s % 8] != b'H':
                return UP
            else:
                return LEFT
        else:
            if s // 8 != 7 and env.desc[s // 8 + 1, s % 8] != b'H':
                return DOWN
            elif s % 8 != 7 and env.desc[s // 8, s % 8 + 1] != b'H':
                return RIGHT
            elif s % 8 != 0 and env.desc[s // 8, s % 8 - 1] != b'H':
                return LEFT
            else:
                return UP

    V = np.where(env.desc == b'H', -1, 1).reshape(64).astype('float64')
    np.set_printoptions(precision=4)
    print(td_lambtha(env, V, policy, 0.9).reshape((8, 8)))

[[-0.861  -0.8645 -0.8442 -0.8304 -0.8075 -0.6999 -0.6783 -0.6727]
 [-0.8876 -0.8846 -0.8883 -0.887  -0.8548 -0.7688 -0.6772 -0.5907]
 [-0.9118 -0.9322 -0.9507 -1.     -0.9299 -0.8631 -0.7866 -0.6296]
 [-0.9261 -0.9348 -0.9436 -0.9843 -0.978  -1.     -0.811  -0.8148]
 [-0.9547 -0.9688 -0.9571 -1.     -0.9501 -0.8894 -0.8801 -0.8171]
 [-0.9704 -1.     -1.      0.3269 -0.9353 -0.8666 -1.     -0.3871]
 [-0.9711 -1.     -0.5221  0.0472 -1.     -0.7219 -1.      0.8954]
 [-0.9355 -0.8799 -0.8341 -1.      1.     -0.2502  0.801   1.    ]]


## 2. SARSA(λ)
<br>
Write the function def sarsa_lambtha(env, Q, lambtha, episodes=5000, max_steps=100, alpha=0.1, gamma=0.99, epsilon=1, min_epsilon=0.1, epsilon_decay=0.05): that performs SARSA(λ):

env is the openAI environment instance<br>
Q is a numpy.ndarray of shape (s,a) containing the Q table<br>
lambtha is the eligibility trace factor<br>
episodes is the total number of episodes to train over<br>
max_steps is the maximum number of steps per episode<br>
alpha is the learning rate<br>
gamma is the discount rate<br>
epsilon is the initial threshold for epsilon greedy<br>
min_epsilon is the minimum value that epsilon should decay to<br>
epsilon_decay is the decay rate for updating epsilon between episodes<br>
Returns: Q, the updated Q table<br>
```
$ ./2-main.py
[[0.5452 0.5363 0.6315 0.5329]
 [0.5591 0.6166 0.5316 0.5425]
 [0.5336 0.602  0.529  0.5463]
 [0.5475 0.5974 0.5362 0.5436]
 [0.5531 0.5693 0.6117 0.568 ]
 [0.6147 0.6011 0.6511 0.5966]
 [0.6472 0.6183 0.599  0.6176]
 [0.6334 0.6267 0.6519 0.634 ]
 [0.5571 0.5233 0.646  0.5867]
 [0.6456 0.5602 0.545  0.5321]
 [0.6303 0.53   0.5055 0.5394]
 [0.4495 0.4853 0.4384 0.5781]
 [0.5291 0.5351 0.5489 0.5821]
 [0.6182 0.6166 0.6186 0.6047]
 [0.6266 0.5832 0.6497 0.5645]
 [0.5369 0.3657 0.7081 0.4936]
 [0.5924 0.7393 0.5806 0.5818]
 [0.5621 0.7052 0.5681 0.5429]
 [0.6894 0.509  0.4663 0.5361]
 [0.2828 0.1202 0.2961 0.1187]
 [0.4457 0.4633 0.411  0.5208]
 [0.5899 0.6983 0.7595 0.5963]
 [0.7263 0.699  0.6698 0.6954]
 [0.6126 0.7508 0.4898 0.4768]
 [0.6615 0.5872 0.7568 0.5987]
 [0.5805 0.5433 0.5839 0.7284]
 [0.6236 0.6239 0.7243 0.5689]
 [0.6498 0.7383 0.6077 0.5422]
 [0.6334 0.6377 0.7003 0.6311]
 [0.8811 0.5813 0.8817 0.6925]
 [0.7557 0.7478 0.7796 0.7706]
 [0.6687 0.8253 0.65   0.5062]
 [0.6277 0.7568 0.6078 0.6561]
 [0.6366 0.6973 0.6338 0.7487]
 [0.7121 0.7965 0.7082 0.7455]
 [0.8965 0.3676 0.4359 0.8919]
 [0.7498 0.8535 0.3625 0.7401]
 [0.7681 0.7448 0.2974 0.837 ]
 [0.4996 0.6835 0.4382 0.8703]
 [0.8936 0.7053 0.4904 0.3181]
 [0.6677 0.7224 0.8078 0.6766]
 [0.9755 0.8558 0.0117 0.36  ]
 [0.73   0.1716 0.521  0.0543]
 [0.2466 0.0813 0.8518 0.2852]
 [0.3454 0.8602 0.7229 0.1075]
 [0.2801 0.7741 0.6684 0.288 ]
 [0.9342 0.614  0.5356 0.5899]
 [1.0137 0.391  0.4284 0.2431]
 [0.382  0.4696 0.4571 0.599 ]
 [0.2274 0.2544 0.058  0.4344]
 [0.3118 0.6755 0.4197 0.1796]
 [0.0247 0.0672 0.778  0.4537]
 [0.5366 0.8967 0.9903 0.2169]
 [0.6914 0.3132 0.0996 0.7817]
 [0.32   0.3835 0.5883 0.831 ]
 [0.629  1.3232 0.2735 0.8131]
 [0.2803 0.5022 0.5382 0.2851]
 [0.6295 0.6324 0.2997 0.2133]
 [0.5699 0.0643 0.2075 0.4247]
 [0.3742 0.4636 0.2776 0.5868]
 [0.8639 0.1175 0.5174 0.1321]
 [0.7169 0.3961 0.5654 0.1833]
 [0.1448 0.4881 0.3556 0.9404]
 [0.7653 0.7487 0.9037 0.0834]]
 ```

In [ ]:
def select_action(state, Q, epsilon):
    """
    select an action based on the epsilon-greedy strategy

    params:
    state: The current state of the environment.
    Q (numpy.ndarray): Q-table, containing the Q-values for each state-action pair
    epsilon: probability of choosing a random action (exploration)

    returns the selected action
    """
    # if a random number is less than epsilon, pick a random action for exploration
    #   otherwise, pick the best action from Q-table for exploitation
    if np.random.uniform() < epsilon:
        return np.random.randint(0, Q.shape[1])
    else:
        return np.argmax(Q[state, :])

def update_epsilon(episode, min_epsilon, epsilon_decay):
    """
    update epsilon value for the epsilon-greedy strategy

    params:
    episode: The current episode number.
    min_epsilon: minimum value to which epsilon can decay
    epsilon_decay: decay rate for epsilon

    returns te updated epsilon value
    """
    # update the value of epsilon after each episode
    #   this gradually reduces epsilon, shifting from exploration to exploitation
    # uses exponential decay for a smooth transition
    return min_epsilon + (1 - min_epsilon) * np.exp(-1 * epsilon_decay * episode)

def calculate_TD_error(reward, gamma, Q, state, action, next_state, next_action):
    """
    calculate TD error (temporal difference)

    params:
    reward: reward received after taking the action
    gamma: discount factor
    Q (numpy.ndarray): Q-table
    state: current state
    action: action taken in the current state
    next_state: next state after taking the action
    next_action: The action taken in the next state

    returns the calculated TD error
    """
    # TD error is the difference between the estimated value and the observed value
    #   it's a measure of how good the Q-value prediction was
    return reward + gamma * Q[next_state, next_action] - Q[state, action]

def sarsa_lambda(env, Q, lambda_value, episodes=5000, max_steps=100, alpha=0.1, gamma=0.99, epsilon=1, min_epsilon=0.1, epsilon_decay=0.05):
    """
    performs SARSA(λ)

    env is the openAI environment instance
    Q is a numpy.ndarray of shape (s,a) containing the Q table
    lambtha is the eligibility trace factor
    episodes is the total number of episodes to train over
    max_steps is the maximum number of steps per episode
    alpha is the learning rate
    gamma is the discount rate
    epsilon is the initial threshold for epsilon greedy
    min_epsilon is the minimum value that epsilon should decay to
    epsilon_decay is the decay rate for updating epsilon between episodes

    Returns: Q, the updated Q table
    """
    # eligibility traces are used to track the 'eligibility' of each state-action pair for learning
    eligibilityTraces = np.zeros_like(Q)

    # iterate over each episode
    #   (an episode is a sequence of states, actions, and rewards, which ends with a terminal state)
    for episode in range(episodes):
        # reset environment for this episode
        state = env.reset()

        # choose the first action
        action = select_action(state, Q, epsilon)

        for step in range(max_steps):
            # take the action and observe the outcome
            next_state, reward, terminate, _ = env.step(action)

            # select next action using the epsilon-greedy policy, based on the new state
            next_action = select_action(next_state, Q, epsilon)

            # calculate the TD error and update the Q-table and eligibility traces
            #   (represents the difference between the estimated value (Q-value)
            #     and the observed value (reward + value of next state))
            delta = calculate_TD_error(reward, gamma, Q, state, action, next_state, next_action)

            # increase the eligibility trace for the current state-action pair
            #  marks the state-action pair as eligible for learning
            eligibilityTraces[state, action] += 1

            # update Q-table using the TD error, learning rate (alpha), and eligibility traces
            #  (adjusts the Q-value of the state-action pair, moving it closer to the expected long-term return)
            Q += alpha * delta * eligibilityTraces

            # decay the eligibility traces for all state-action pairs
            #   (reduces the eligibility of state-action pairs over time, with a rate controlled by lambda_value and gamma)
            #   (ensures that only recent state-action pairs have a significant effect on learning)
            eligibilityTraces *= lambda_value * gamma

            # update the current state and action to the next state and action,
            #  moving the agent through the environment
            state, action = next_state, next_action

            # if the agent reached a terminal state exit loop for current episode
            if terminate:
                break

        # update the exploration rate (epsilon) using the decay function
        epsilon = update_epsilon(episode, min_epsilon, epsilon_decay)

    # this Q-table represents the learned values for each state-action pair
    return Q

if __name__ == "__main__":
    import gym
    import numpy as np

    np.random.seed(0)
    env = gym.make('FrozenLake8x8-v1')
    Q = np.random.uniform(size=(64, 4))
    np.set_printoptions(precision=4)
    print(sarsa_lambtha(env, Q, 0.9))

/usr/local/lib/python3.10/dist-packages/gym/core.py:317: DeprecationWarning: WARN: Initializing wrapper in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(
/usr/local/lib/python3.10/dist-packages/gym/wrappers/step_api_compatibility.py:39: DeprecationWarning: WARN: Initializing environment in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(


[[0.577  0.5661 0.7281 0.6128]
 [0.6914 0.5475 0.5336 0.6137]
 [0.6633 0.584  0.5316 0.5574]
 [0.5126 0.578  0.528  0.6215]
 [0.5814 0.6064 0.6452 0.6049]
 [0.6298 0.6188 0.7422 0.6403]
 [0.6588 0.6868 0.583  0.6272]
 [0.6272 0.7334 0.5934 0.6229]
 [0.7643 0.5776 0.5977 0.5683]
 [0.5646 0.6838 0.5756 0.5467]
 [0.5535 0.6873 0.5522 0.5964]
 [0.4974 0.578  0.4594 0.665 ]
 [0.5942 0.6644 0.6176 0.6292]
 [0.7087 0.6393 0.676  0.62  ]
 [0.6742 0.6645 0.7151 0.5975]
 [0.5651 0.4632 0.7753 0.5404]
 [0.5894 0.5998 0.7609 0.5806]
 [0.7465 0.6096 0.5601 0.5595]
 [0.7187 0.5315 0.4977 0.5087]
 [0.2828 0.1202 0.2961 0.1187]
 [0.4475 0.6715 0.463  0.4508]
 [0.7117 0.7488 0.8323 0.659 ]
 [0.752  0.8477 0.6777 0.6856]
 [0.6386 0.8721 0.6041 0.6138]
 [0.6618 0.8579 0.627  0.7004]
 [0.6423 0.8141 0.5846 0.6123]
 [0.603  0.771  0.6075 0.6504]
 [0.6116 0.7795 0.6068 0.6504]
 [0.6265 0.8585 0.6318 0.6311]
 [0.8811 0.5813 0.8817 0.6925]
 [0.8842 0.7772 0.735  0.7611]
 [0.7077 0.897  0.5979 0.6868]
 [0.6308